In [2]:
!pip install pyradiomics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.5/34.5 MB 45.8 MB/s eta 0:00:00:00:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Discarding https://files.pythonhosted.org/packages/03/c1/20fc2c50ab1e3304da36d866042a1905a2b05a1431ece35448ab6b4578f2/pyradiomics-3.1.0.tar.gz (from https://pypi.org/simple/pyradiomics/): Requested pyradiomics from https://files.pythonhosted.org/packages/03/c1/20fc2c50ab1e3304da36d866042a1905a2b05a1431ece35448ab6b4578f2/pyradiomics-3.1.0.tar.gz has inconsistent version: expected '3.1.0', but metadata has '3.0.1a1'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.5/34.5 MB 40.5 MB/s eta 0:00:00:00:010:01
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.8/117.8 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 722.2/722.2 kB 42.6 MB/s eta 0:00:00
  Created wheel for p

In [3]:
import numpy as np 
import pandas as pd 
import os
import cv2
import matplotlib.pyplot as plt
from skimage import io, feature
from skimage.feature import graycomatrix, graycoprops
import radiomics
from radiomics import featureextractor
import SimpleITK as sitk

In [5]:
train=pd.read_csv("/kaggle/input/combined-mpox/train_aug.csv")
train["type"]="train"
train=train.drop(columns=["Unnamed: 0"])
train

,filename,paths,labels,source,type
0,HFMD_16_02.jpg,/kaggle/input/mpox-skin-lesion-dataset-version...,HFMD,MSLD,train
1,chickenpox101.png,/kaggle/input/monkeypoxskinimagedataset/Monkey...,chpox,MSID,train
2,normal209.png,/kaggle/input/monkeypoxskinimagedataset/Monkey...,normal,MSID,train
3,chickenpox66.png,/kaggle/input/monkeypoxskinimagedataset/Monkey...,chpox,MSID,train
4,HEALTHY_31_01.jpg,/kaggle/input/mpox-skin-lesion-dataset-version...,normal,MSLD,train
...,...,...,...,...,...
6976,aug_0_6998.jpeg,/kaggle/input/combined-mpox/augment_folder/nor...,normal,NaN,train
6977,aug_0_1044.jpeg,/kaggle/input/combined-mpox/augment_folder/nor...,normal,NaN,train
6978,aug_0_4681.jpeg,/kaggle/input/combined-mpox/augment_folder/nor...,normal,NaN,train
6979,aug_0_9956.jpeg,/kaggle/input/combined-mpox/augment_folder/nor...,normal,NaN,train


In [6]:
test=pd.read_csv("/kaggle/input/combined-mpox/augment_folder/test.csv")
test["type"]="test"
test

,filename,paths,labels,source,type
0,CWP_33_04.jpg,/kaggle/input/mpox-skin-lesion-dataset-version...,CWP,MSLD,test
1,CWP_37_09.jpg,/kaggle/input/mpox-skin-lesion-dataset-version...,CWP,MSLD,test
2,MSL_17_01.jpg,/kaggle/input/mpox-skin-lesion-dataset-version...,measles,MSLD,test
3,HFMD_03_01.jpg,/kaggle/input/mpox-skin-lesion-dataset-version...,HFMD,MSLD,test
4,monkeypox37.png,/kaggle/input/monkeypoxskinimagedataset/Monkey...,mpox,MSID,test
...,...,...,...,...,...
286,chickenpox67.png,/kaggle/input/monkeypoxskinimagedataset/Monkey...,chpox,MSID,test
287,normal45.png,/kaggle/input/monkeypoxskinimagedataset/Monkey...,normal,MSID,test
288,normal287.png,/kaggle/input/monkeypoxskinimagedataset/Monkey...,normal,MSID,test
289,chickenpox19.png,/kaggle/input/monkeypoxskinimagedataset/Monkey...,chpox,MSID,test


In [10]:
MSD=pd.concat([train,test])
MSD.reset_index(inplace=True)

In [8]:
csv_file = "/kaggle/input/combined-mpox/MSD.csv" 
data = MSD
results = []
for index, row in data.iterrows():
    image_path = row["paths"]
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    img_quantized = (img // (256 // 16)).astype('uint8')
    glcm = graycomatrix(img_quantized, distances=[1], angles=[0], levels=16, symmetric=True, normed=True)
    contrast = graycoprops(glcm, 'contrast').item()
    energy = graycoprops(glcm, 'energy').item()
    homogeneity = graycoprops(glcm, 'homogeneity').item()
    correlation = graycoprops(glcm, 'correlation').item()

    results.append({
            "Image": row['labels'],
            "Contrast": contrast,
            "Energy": energy,
            "Homogeneity": homogeneity,
            "Correlation": correlation
        })
    
    GLCM = pd.DataFrame(results)
print(GLCM)


       Image  Contrast    Energy  Homogeneity  Correlation
0       HFMD  0.210147  0.301620     0.906171     0.985553
1      chpox  0.117593  0.389789     0.943291     0.988266
2     normal  0.613429  0.556937     0.907036     0.970025
3      chpox  0.372257  0.222388     0.867223     0.984544
4     normal  0.647206  0.258355     0.861055     0.969472
...      ...       ...       ...          ...          ...
7267   chpox  0.278547  0.309786     0.886471     0.976524
7268  normal  0.486227  0.440476     0.885155     0.986149
7269  normal  0.467249  0.254317     0.879532     0.974878
7270   chpox  0.124099  0.568271     0.942602     0.978876
7271    HFMD  0.269458  0.290857     0.908303     0.978541

[7272 rows x 5 columns]


In [11]:
GLCM["paths"]=MSD["paths"]
GLCM["labels"]=MSD["labels"]
GLCM["type"]=MSD["type"]

In [12]:
GLCM.to_csv("GLCM.csv",index=False)

In [13]:
results = []
for index, row in data.iterrows():
    image_path = row["paths"]
    image = sitk.ReadImage(image_path, sitk.sitkFloat32)    
    mask = sitk.OtsuThreshold(image)
    
    extractor = featureextractor.RadiomicsFeatureExtractor()
    extractor.enableFeatureClassByName('shape2D')
    extractor.enableFeatureClassByName('glrlm')  
    features = extractor.execute(image, mask)
    GLRLM_list = {"Image": row['labels']}
    
    for feature_name, value in features.items():
        if 'glrlm' in feature_name:
            GLRLM_list[feature_name] = float(value)  
    
    results.append(GLRLM_list)
    GLRLM = pd.DataFrame(results)
print(GLRLM)

Shape features are only available 3D input (for 2D input, use shape2D). Found 2D input
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
Shape features are only available 3D input (for 2D input, use shape2D). Found 2D input
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
Shape features are only available 3D input (for 2D input, use shape2D). Found 2D input
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
Shape features are only available 3D input (for 2D input, use shape2D). Found 2D input
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
Shape features are only available 3D input (for 2D input, use shape2D). Found 2D input
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
Shape features are only available 3D input (for 2D input, use shape2D). Found 2D input
GLCM is 

       Image  original_glrlm_GrayLevelNonUniformity  \
0       HFMD                             920.864214   
1      chpox                             303.114385   
2     normal                             655.050904   
3      chpox                             999.984066   
4     normal                             737.443775   
...      ...                                    ...   
7267   chpox                             900.084382   
7268  normal                             941.357205   
7269  normal                            1225.435664   
7270   chpox                             138.659588   
7271    HFMD                            3273.013580   

      original_glrlm_GrayLevelNonUniformityNormalized  \
0                                            0.254635   
1                                            0.205252   
2                                            0.212930   
3                                            0.200600   
4                                            0.201339 

In [14]:
GLRLM["paths"]=MSD["paths"]
GLRLM["labels"]=MSD["labels"]
GLRLM["type"]=MSD["type"]
GLRLM.to_csv("GLRLM.csv",index=False)


In [ ]:
# result=[]
# for index, row in data.iterrows():
#     image_path = row["paths"]
#     image = sitk.ReadImage(image_path, sitk.sitkFloat32)  
#     extractor = featureextractor.RadiomicsFeatureExtractor()
#     extractor = featureextractor.RadiomicsFeatureExtractor(binWidth=20, sigma=[1, 2, 3], verbose=True)
#     extractor.disableAllFeatures()
#     extractor.enableFeatureClassByName('glrlm')
#     glrlm_features = glrlm.RadiomicsGLRLM.getFeatureNames()
#     extractor.enableFeaturesByName(glrlm=glrlm_features)
#     mask_array = np.ones(sitk.GetArrayFromImage(image).shape, dtype=np.uint8)
#     mask_array[0, :] = 0  
#     mask_array[-1, :] = 0  
#     mask_array[:, 0] = 0  
#     mask_array[:, -1] = 0  
#     mask_with_borders_zeroed = sitk.GetImageFromArray(mask_array)
#     mask_with_borders_zeroed.CopyInformation(image)
    
#     features = extractor.execute(image, mask_with_borders_zeroed)
#     GLRLM_list = {"Image": row['labels']}
        
#     for feature_name, value in features.items():
#         if 'glrlm' in feature_name:
#             GLRLM_list[feature_name] = value
        
#     results.append(GLRLM_list)
#     GLRLM_whole = pd.DataFrame(results)
# print(GLRLM_whole)

In [15]:
results = []
for index, row in data.iterrows():
    image_path = row["paths"]
    image = sitk.ReadImage(image_path, sitk.sitkFloat32)
    mask = sitk.OtsuThreshold(image)

    extractor = featureextractor.RadiomicsFeatureExtractor()
    extractor.enableFeatureClassByName('shape2D')
    extractor.enableFeatureClassByName('glszm')  
    features = extractor.execute(image, mask)
    GLSZM_list = {"Image": row['labels']}
        
    for feature_name, value in features.items():
        if 'glszm' in feature_name:
            GLSZM_list[feature_name] = float(value)  
        
    results.append(GLSZM_list)

GLSZM = pd.DataFrame(results)

print(GLSZM)


Shape features are only available 3D input (for 2D input, use shape2D). Found 2D input
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
Shape features are only available 3D input (for 2D input, use shape2D). Found 2D input
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
Shape features are only available 3D input (for 2D input, use shape2D). Found 2D input
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
Shape features are only available 3D input (for 2D input, use shape2D). Found 2D input
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
Shape features are only available 3D input (for 2D input, use shape2D). Found 2D input
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
Shape features are only available 3D input (for 2D input, use shape2D). Found 2D input
GLCM is 

       Image  original_glszm_GrayLevelNonUniformity  \
0       HFMD                              40.578947   
1      chpox                              64.867550   
2     normal                             117.267686   
3      chpox                             165.826923   
4     normal                              92.313559   
...      ...                                    ...   
7267   chpox                             187.368889   
7268  normal                              98.751227   
7269  normal                             218.864943   
7270   chpox                              18.632653   
7271    HFMD                             273.107969   

      original_glszm_GrayLevelNonUniformityNormalized  \
0                                            0.237304   
1                                            0.429586   
2                                            0.224221   
3                                            0.318898   
4                                            0.195580 

In [16]:
GLSZM["paths"]=MSD["paths"]
GLSZM["labels"]=MSD["labels"]
GLSZM["type"]=MSD["type"]
GLSZM.to_csv("GLSZM.csv",index=False)

In [ ]:
GLSZM.info()

In [ ]:
def process_img(img):
    img=cv2.imread(img)
    resized=cv2.resize(img,(224,224),interpolation=cv2.INTER_NEAREST)
    image = cv2.normalize(resized, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    image = clahe.apply(img)
    return image

https://stackoverflow.com/questions/65825365/pyradiomics-how-to-extract-features-from-gray-level-run-length-matrix-using-pyr
https://pyradiomics.readthedocs.io/en/latest/features.html#module-radiomics.shape2D

In [ ]:
encodGLCM = pd.get_dummies(GLCM, columns=['Image'], drop_first=False, dtype=int)
encodGLRLM = pd.get_dummies(GLRLM, columns=['Image'], drop_first=False, dtype=int)
encodGLSZM = pd.get_dummies(GLSZM, columns=['Image'], drop_first=False, dtype=int)

In [ ]:
def correlation(dataset, threshold):
    correlated_pairs = []  
    corr_matrix = dataset.corr()  
    for i in range(len(corr_matrix.columns)):
        for j in range(i): 
            if abs(corr_matrix.iloc[i, j]) > threshold:
                col1 = corr_matrix.columns[i]
                col2 = corr_matrix.columns[j]
                correlated_pairs.append((col1, col2, corr_matrix.iloc[i, j]))  
    
    return correlated_pairs

correlated_columns = correlation(encodGLRLM+encodGLSZM, 0.8)
if correlated_columns:
    print("Correlated column pairs (with correlation values):")
    for col1, col2, corr_value in correlated_columns:
        print(f"{col1} - {col2}: {corr_value:.2f}")
else:
    print("No correlated columns found.")


In [ ]:
GLCM = GLCM.drop(columns=['Homogeneity'])
GLRLM = GLRLM.drop(columns=['original_glrlm_HighGrayLevelRunEmphasis','original_glrlm_RunPercentage','original_glrlm_RunLengthNonUniformityNormalized','original_glrlm_RunVariance','original_glrlm_GrayLevelNonUniformity','original_glrlm_LongRunLowGrayLevelEmphasis'])
GLSZM = GLSZM.drop(columns=['original_glszm_LargeAreaLowGrayLevelEmphasis','original_glszm_SizeZoneNonUniformity','original_glszm_SizeZoneNonUniformityNormalized','original_glszm_SmallAreaHighGrayLevelEmphasis','original_glszm_LargeAreaEmphasis','original_glszm_LargeAreaLowGrayLevelEmphasis'])

In [ ]:
csvFile = GLCM.to_csv('GLCM.csv', index = False) 
csvFile = GLRLM.to_csv('GLRLM.csv', index = False) 
csvFile = GLSZM.to_csv('GLSZM.csv', index = False) 